# Expore and Cleanup
and also add some features

In [ ]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns 
import sys
sys.path.append('../Functions')
import CleanUp as p

In [ ]:
file_path = '../Data/DM2425_ABCDEats_DATASET.csv'
df = pd.read_csv(file_path)
shape_of_df = df.shape
shape_of_df

## 1 Exploration

In [ ]:
df.head()

### 1.1 count the Nans

In [ ]:
number_of_nans = df.isnull().sum()
x_xis = number_of_nans.index.tolist()
plt.figure(figsize=(10, 5))
plt.xticks(rotation=90)
plt.ylabel('Number of NaNs in raw data')
plt.title('Number of NaNs in each column')
plt.bar(x=x_xis, height=number_of_nans, color='mediumaquamarine', edgecolor='dimgray', linewidth=0.8)

plt.show()

In [ ]:
number_of_nans.sum()

### 1.2 Check for Duplicates in the primary key

by the shape of the dataset the primary key should be customer_id, as such every value in this column should be unique

In [ ]:
primary_key = 'customer_id'
unique_values = df[primary_key].unique().shape
n_rows = df[primary_key].shape [0]
if unique_values != n_rows:
    print('oh no! there are repetitions!')
else:
    print("all good! no repetitions!")

In [ ]:
#find repeated customer_id
repetitions = df[primary_key].value_counts()
#keep only the customer_id that are repeated
repetitions = repetitions[repetitions > 1]
repetitions = repetitions.index
repetitions

In [ ]:
for customer_id in repetitions:
    rep_df = df[df[primary_key] == customer_id]
    s = rep_df.shape
    s1 = 0
    for i in range(s[1]):
        s1 += rep_df.iloc[:,i].unique().shape[0]
    print(s1 - s[1])
    # remove ont of the repeated  primary keys
    if s1 - s[1] == 0:
        df = df.drop(rep_df.index[0])
        print('dropped repeated customer {p}'.format( p = customer_id))   

In [ ]:
df['first_order'].min()

In [ ]:
print("rows: \t{p1} \nclients:{p2}".format( p1 = df.shape [0],
                                        p2 = df[primary_key].unique().shape[0]))


## 2 Non Numerical Columns

In [ ]:
non_numerical_columns = df.select_dtypes(include=['object']).columns.tolist()
non_numerical_columns.remove(primary_key)
print(non_numerical_columns)

In [ ]:
for col in non_numerical_columns:
    print(f'{col}: {df[col].unique()}')

### 2.1 customer_region
Geographic region where the customer is located.

In [ ]:
df['customer_region'].unique()  

#### 2.1.1 deal with missing values 

In [ ]:
df.loc[df['customer_region'] == '-', 'customer_region'] = np.nan

In [ ]:
df['customer_region'].unique()  

#### 2.1.2 exploration

In [ ]:
region_count_dis = df['customer_region'].value_counts()
region_percentage = region_count_dis / region_count_dis.sum() * 100
print(region_percentage)
region_nan = df['customer_region'].isnull().sum()
print(region_nan/df.shape[0]*100)

#order region_nan by alphabetical order
x = zip(list(region_count_dis.index),list(region_count_dis.values))
x = sorted(x, key=lambda x: x[0])
region_count = dict(x)
region_count

In [ ]:
# Adjusted colors
colors = ['lightsalmon', 'mediumaquamarine', 'tomato','lightsalmon', 'mediumaquamarine','tomato' ,'mediumaquamarine',  'tomato']

# Configuring the pie chart with labels and highlighting the most relevant slice
plt.figure(figsize=(20, 10))  # Setting figure size
plt.pie(region_count_dis,
        labels=region_count.keys(),
        colors=colors,
        startangle=90,  # Rotate the start of the pie chart
        wedgeprops={'edgecolor': 'white', 'linewidth': 2})  # Creating the pie chart with black and thick edges
plt.title('Regions distribution')  # Setting the title
plt.show()  # Displaying the plot


In [ ]:
# Adjusted colors
colors = ['lightsalmon', 'lightsalmon', 'lightsalmon', 'mediumaquamarine', 'mediumaquamarine', 'tomato', 'tomato', 'tomato']

# Configuring the pie chart with labels and highlighting the most relevant slice
plt.figure(figsize=(20, 10))  # Setting figure size
plt.pie(region_count.values(),
        labels=region_count.keys(),
        colors=colors,
        startangle=90,  # Rotate the start of the pie chart
        wedgeprops={'edgecolor': 'white', 'linewidth': 2})  # Creating the pie chart with black and thick edges
plt.title('Regions distribution')  # Setting the title
plt.show()  # Displaying the plot


In [ ]:

sns.barplot(region_count_dis,color='mediumaquamarine')

### 2.2 last_promo
The category of the promotion or discount most recently used by the
custome

#### 2.2.1 deal with missing values 

In [ ]:
df['last_promo'].unique()

In [ ]:
# make a new boolean column the is called has_promo 

df['has_promo'] = df['last_promo'] != 'No_Promo'
df['has_promo'].describe()

#### 2.2.2 exploration

In [ ]:
# Adjusted colors
colors = ['lightsalmon', 'mediumaquamarine', 'tomato', 'darkturquoise']

# Configuring the pie chart with labels and highlighting the most relevant slice
plt.figure(figsize=(10, 5))  # Setting figure size
plt.pie(df.last_promo.value_counts(), 
        colors=colors, 
        labels=df.last_promo.value_counts().index,  # Labels for each slice
        autopct='%1.1f%%',  # Percentages on each slice
        startangle=140,  # Starting angle for better layout
        explode=(0.1, 0, 0, 0),  # Highlight the first slice (adjust as needed)
        wedgeprops={'edgecolor': 'gray'})  # Gray borders for contrast

# Chart title
plt.title("Promotion Distribution", fontsize=16, fontweight='bold', color='darkslategray', pad=20)

# Displaying the chart
plt.tight_layout()
plt.show()

### 2.3 payment_method

In [ ]:
df['payment_method'].unique()

In [ ]:
total_pay = df['payment_method'].value_counts()
plt.figure(figsize=(10,10))
plt.bar(x=total_pay.index, height=total_pay.values, color='mediumaquamarine', edgecolor='dimgray', linewidth=0.8)
plt.title('Payment Method Distribution')
plt.show()

In [ ]:
total_pay

## 3 Numerical Columns

In [ ]:
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(numerical_columns)

### 3.1 vendor_count

In [ ]:
df['vendor_count'].describe()

In [ ]:
plt.hist(df['vendor_count'], bins = 50, color = 'mediumaquamarine', edgecolor = 'dimgray')

plt.xlabel('Vendor Count')
plt.ylabel('Frequency')
plt.title('Vendor Count Distribution')
#adda  line to shw the outliers
plt.axvline(df['vendor_count'].mean(), color='k', linestyle='dashed', linewidth=1)
Q1 = df['vendor_count'].quantile(0.25)
Q3 = df['vendor_count'].quantile(0.75)
IQR = Q3 - Q1
plt.axvline(Q3 + 1.5 * IQR, color='r', linestyle='dashed', linewidth=1)
plt.show()


### 3.2 product_count
Total number of products the customer has ordered.

In [ ]:
plt.hist(df['product_count'], bins = 50, color = 'mediumaquamarine', edgecolor = 'dimgray')

plt.xlabel('product Count')
plt.ylabel('Frequency')
plt.title('product Count Distribution')
#adda  line to shw the outliers
plt.axvline(df['product_count'].mean(), color='k', linestyle='dashed', linewidth=1)
Q1 = df['product_count'].quantile(0.25)
Q3 = df['product_count'].quantile(0.75)
IQR = Q3 - Q1
plt.axvline(Q3 + 1.5 * IQR, color='r', linestyle='dashed', linewidth=1)
plt.show()

### 3.3 is_chain
Indicates whether the customer’s order was from a chain restaurant.

### 3.3.1 exploration

In [ ]:
df['is_chain'].describe()

In [ ]:
plt.hist(df['is_chain'], bins = 50, color = 'mediumaquamarine', edgecolor = 'dimgray')

plt.xlabel('is_chain')
plt.ylabel('Frequency')
plt.title('is_chain Distribution')
#adda  line to shw the outliers
plt.axvline(df['is_chain'].mean(), color='k', linestyle='dashed', linewidth=1)
Q1 = df['is_chain'].quantile(0.25)
Q3 = df['is_chain'].quantile(0.75)
IQR = Q3 - Q1
plt.axvline(Q3 + 1.5 * IQR, color='r', linestyle='dashed', linewidth=1)
plt.show()

#### 3.3.2 check for consistency with product_count and vendor_count

In [ ]:
print("product_count-->",(df['product_count'] - df['is_chain']).min())
print("vendor_count-->",(df['vendor_count'] - df['is_chain']).min())

## 3 Recount the missing values after the mild preprocessing

In [ ]:
#check for missing values
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
number_of_nans = df.isnull().sum()
x_xis = number_of_nans.index.tolist()
plt.figure(figsize=(10, 5))
plt.xticks(rotation=90)
plt.ylabel('Number of NaNs mild ')
plt.title('Number of NaNs in each column')
plt.bar(x=x_xis, height=number_of_nans, color='mediumaquamarine', edgecolor='dimgray', linewidth=0.8)

plt.show()

FYI,there are missing values

## 3 Build New Features

In [ ]:
new_fetures_list = []
# is there  a naming convention for new features?

### 3.1 customer_city

In [ ]:
df['customer_city'] = df['customer_region'].str[0]
df['customer_city'].unique()
new_fetures_list.append('customer_city')

In [ ]:
city_count = df['customer_city'].value_counts()
city_percentage = city_count / city_count.sum() * 100
print(city_percentage)

In [ ]:
plt.figure(figsize=(10,10))
colors = ['lightsalmon', 'mediumaquamarine', 'tomato']
plt.pie(city_percentage, labels = city_percentage.index, autopct='%1.1f%%', colors= colors)
plt.title('City distribution')
plt.show()

In [ ]:
sns.barplot(city_count,color='mediumaquamarine')

### 3.2 used_promo

In [ ]:
df['used_promo']= df['last_promo'] != '-'
new_fetures_list.append("used_promo")

### 3.3 order_count
The total amount of orders each customer made. <br>
it can either be calculated by summung the DOW orders or the HR orders, let's see if the result is consistent <br>
as we have seen at teh beginning, there are nans in the HR, so it's eay to expecr a discrepancy

In [ ]:
DOW_col = [col for col in df.columns if col.startswith('DOW')]
HR_col = [col for col in df.columns if col.startswith('HR')]
DOW_col_sum = df[DOW_col].sum(axis=1)
HR_col_sum = df[HR_col].sum(axis=1)
Delta_DOW_HR = DOW_col_sum-HR_col_sum
Delta_DOW_HR.value_counts()

the values are not consistent <br>
But we can notice that the values are always positive! <br>
Since there are also missing values in HR, it's possible to assume that HR is underestimating and we can fill the missing values with the difference
meanwhile the new feature Total_Orders will be made using the DOW data

In [ ]:
df['order_count'] = DOW_col_sum
HR_col_from1 = HR_col.copy()
HR_col_from1.remove('HR_0')
HR_col_from1_sum = df[HR_col_from1].sum(axis=1)
df['HR_0'] = DOW_col_sum - HR_col_from1_sum

df[HR_col].sum().sum() - df[DOW_col].sum().sum()

#### check for consistency with is_chain

In [ ]:
print("order_count -->",(df['order_count'] - df['is_chain']).min())

there are cases that are values of is_chain tha are bigger than teh number of orders and product count or the vendor count, this gives problem reagrding teh interpretation

### 3.4 avg_product_by_Order

In [ ]:
df['avg_product_by_order'] = p.safe_divide(df['product_count'],df['order_count'])
new_fetures_list.append("avg_product_by_order")

### 3.5 delta_day_order
shows the time passed beween the first order and the last order
-- note:
we could adjust the parameter by using the highest day for the last order in the dataframe as today

In [ ]:

df['delta_day_order'] = df['last_order'] - df['first_order'] + 1
new_fetures_list.append('delta_day_order')

### 3.6 tot_value_cui
shows the total amunt of money spent on the app across all options

In [ ]:
cui_columns = [col for col in df.columns if col.startswith('CUI')]
df['tot_value_cui'] = df[cui_columns].sum(axis=1)
new_fetures_list.append('tot_value_cui')

### 3.7 order_freq
more insightful informations about our customers habits

In [ ]:
df['order_freq'] = p.safe_divide(df['order_count'], df['delta_day_order'])
new_fetures_list.append('order_freq')


### 3.8 value_freq


In [ ]:
df['value_freq'] =p.safe_divide(df['tot_value_cui'], df['delta_day_order'])
new_fetures_list.append('value_freq')


### 3.9 product_freq


In [ ]:
df['product_freq'] = p.safe_divide(df['product_count'], df['delta_day_order'])
new_fetures_list.append('product_freq')


### 3.10 avg_order_value


In [ ]:
df['avg_order_value'] = p.safe_divide(df['tot_value_cui'], df['order_count'])
new_fetures_list.append('avg_order_value')
df['avg_order_value'] = np.where(df['product_count'] != 0, df['tot_value_cui'] / df['product_count'], 0)

### 3.11 avg_product_value


In [ ]:
df['avg_product_value'] = p.safe_divide(df['tot_value_cui'], df['product_count'])
new_fetures_list.append('avg_product_value')


### 3.12 is_chain_bool (not added)

In [ ]:
df['is_chain_bool'] = df['is_chain'] != 0
new_fetures_list.append("is_chain_bool")
x = df[['is_chain_bool','order_count','vendor_count','product_count']]
x_true = x[x['is_chain_bool']]
print(x_true['is_chain_bool'].unique())
print(x_true['order_count'].min())
print(x_true['vendor_count'].min())
print(x_true['product_count'].min())

### 3.13 our new features

In [ ]:
len(new_fetures_list)

In [ ]:
df[new_fetures_list].describe().T

In [ ]:
#check for missing values
missing_values_new_features = df[new_fetures_list].isnull().sum()
missing_values_new_features = [missing_values > 0]

In [ ]:
# plot #check for missing values
missing_values_new_features = df[new_fetures_list].isnull().sum()
missing_values_new_features = missing_values_new_features[missing_values_new_features > 0]
plt.figure(figsize=(10, 5))
plt.bar(missing_values_new_features.index, missing_values_new_features, color='mediumaquamarine', edgecolor='dimgray', linewidth=0.8)
plt.xticks(rotation=90)
plt.ylabel('Number of NaNs')
plt.title('Number of NaNs in each column')

plt.show()

### Check if I made a mess

In [ ]:
nan_inf_counts_list = []

for new_feture in new_fetures_list:
    nan_count = df[new_feture].isna().sum()
    inf_count = df[new_feture].isin([np.inf, -np.inf]).sum()
    nan_inf_counts = {
        'feature': new_feture,
        'nan_count': nan_count,
        'inf_count': inf_count
    }
    
    nan_inf_counts_list.append((new_feture, nan_count, inf_count))
    
df_naninf = pd.DataFrame(nan_inf_counts_list, columns=['feature', 'nan_count', 'inf_count'])
df_naninf

In [ ]:
df[new_fetures_list[0:11]].describe()

In [ ]:
df['customer_city'].describe()

In [ ]:
df['used_promo'].describe()